# 创建Tensor
在PyTorch中，`torch.Tensor`是存储和变换数据的主要工具。如果你之前用过NumPy，你会发现Tensor和NumPy的多维数组非常类似。然而，Tensor提供GPU计算和自动求梯度等更多功能，这些使Tensor更加适合深度学习。

In [2]:
import torch

x = torch.empty(5,3)
print(x)

tensor([[9.1837e-39, 1.0102e-38, 8.4490e-39],
        [1.0102e-38, 9.0919e-39, 1.0102e-38],
        [8.9082e-39, 8.9082e-39, 8.9082e-39],
        [1.0194e-38, 9.1837e-39, 4.6837e-39],
        [9.9184e-39, 9.0000e-39, 1.0561e-38]])


In [3]:
# 创建随机初始化的tensor
x = torch.rand(5,3)
print(x)

tensor([[0.7259, 0.7637, 0.5896],
        [0.5844, 0.1504, 0.9950],
        [0.1298, 0.5107, 0.5530],
        [0.1773, 0.1390, 0.6530],
        [0.8166, 0.3640, 0.8804]])


In [4]:
# long型全0 Tensor
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [5]:
# 直接根据数据创建
x = torch.tensor([8.9, 3])
print(x)

tensor([8.9000, 3.0000])


In [6]:
# 通过现有的Tensor创建 可以重用属性
x = x.new_ones(5, 3, dtype=torch.float64)
print(x)

x = torch.randn_like(x, dtype=torch.float)  # 指定新的类型
print(x)

# 获取tensor形状
print(x.size())
print(x.shape) # 注意：返回的torch.Size其实就是一个tuple, 支持所有tuple的操作。

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[-0.7157,  0.7138,  2.4869],
        [ 1.8405, -0.6086, -1.1210],
        [ 1.1452,  1.8947, -1.7320],
        [-0.4556,  1.3090,  0.3424],
        [ 0.1386, -0.6282, -0.4608]])
torch.Size([5, 3])
torch.Size([5, 3])


# 操作Tensor

## 加法

In [7]:
# 不同的加法形式
# 1
y = torch.rand(5,3)
print(x+y)

# 2
print(torch.add(x,y))

# 3
# 还可以执行运算的输出
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)

# 4
y.add_(x)
print(y)

tensor([[-0.3498,  1.3994,  3.2339],
        [ 2.4827, -0.3416, -0.4119],
        [ 1.2784,  2.0313, -1.4079],
        [-0.3478,  1.4029,  0.6472],
        [ 0.7595, -0.1090,  0.4574]])
tensor([[-0.3498,  1.3994,  3.2339],
        [ 2.4827, -0.3416, -0.4119],
        [ 1.2784,  2.0313, -1.4079],
        [-0.3478,  1.4029,  0.6472],
        [ 0.7595, -0.1090,  0.4574]])
tensor([[-0.3498,  1.3994,  3.2339],
        [ 2.4827, -0.3416, -0.4119],
        [ 1.2784,  2.0313, -1.4079],
        [-0.3478,  1.4029,  0.6472],
        [ 0.7595, -0.1090,  0.4574]])
tensor([[-0.3498,  1.3994,  3.2339],
        [ 2.4827, -0.3416, -0.4119],
        [ 1.2784,  2.0313, -1.4079],
        [-0.3478,  1.4029,  0.6472],
        [ 0.7595, -0.1090,  0.4574]])


## 索引

索引出来的结果与原数据共享内存，也即修改一个，另一个会跟着修改。

In [8]:
y = x[0, :]
y += 1
print(y)
print(x[0, :]) # 源tensor也被改了

tensor([0.2843, 1.7138, 3.4869])
tensor([0.2843, 1.7138, 3.4869])


## 改变形状

In [9]:
y = x.view(15)
z = x.view(-1, 5)
print(y.size(), z.size())

torch.Size([15]) torch.Size([3, 5])


注意view()返回的新Tensor与源Tensor虽然可能有不同的size，但是是共享data的，也即更改其中的一个，另外一个也会跟着改变。(顾名思义，view仅仅是改变了对这个张量的观察角度，内部数据并未改变)
所以如果我们想返回一个真正新的副本（即不共享data内存）该怎么办呢？推荐先用clone创造一个副本然后再使用view。

In [10]:
x_cp = x.clone().view(15)   # 先 clone 再改形状
x -= 1
print(x)
print(x_cp)

tensor([[-0.7157,  0.7138,  2.4869],
        [ 0.8405, -1.6086, -2.1210],
        [ 0.1452,  0.8947, -2.7320],
        [-1.4556,  0.3090, -0.6576],
        [-0.8614, -1.6282, -1.4608]])
tensor([ 0.2843,  1.7138,  3.4869,  1.8405, -0.6086, -1.1210,  1.1452,  1.8947,
        -1.7320, -0.4556,  1.3090,  0.3424,  0.1386, -0.6282, -0.4608])


使用clone还有一个好处是会被记录在计算图中，即梯度回传到副本时也会传到源Tensor。

In [11]:
x = torch.randn(1)
print(x)
print(x.item()) # 另外一个常用的函数就是item(), 它可以将一个标量Tensor转换成一个Python number

tensor([0.6411])
0.6411486268043518


# 线性代数

PyTorch中的Tensor支持超过一百种操作，包括转置、索引、切片、数学运算、线性代数、随机数等等

# 广播机制

两个形状不同的Tensor按元素运算时，可能会触发广播（broadcasting）机制：先适当复制元素使这两个Tensor形状相同后再按元素运算。

In [12]:
x = torch.arange(1, 3).view(1,2)
print(x)
y = torch.arange(1,4).view(3,1)
print(y)
print(x+y)
# x第2、3行复制了第一行
# y第2列复制第一列

tensor([[1, 2]])
tensor([[1],
        [2],
        [3]])
tensor([[2, 3],
        [3, 4],
        [4, 5]])


# 运行时内存
索引操作是不会开辟新内存的，而像`y = x + y`这样的运算是会新开内存的，然后将`y`指向新内存。为了演示这一点，我们可以使用Python自带的id函数：如果两个实例的ID一致，那么它们所对应的内存地址相同；反之则不同

In [13]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
id_before = id(y)
y = y + x
print(id(y) == id_before) # False 

False


如果想指定结果到原来的`y`的内存，我们可以使用前面介绍的索引来进行替换操作。在下面的例子中，我们把`x + y`的结果通过`[:]`写进`y`对应的内存中。

x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
id_before = id(y)
y[:] = y + x
print(id(y) == id_before) # True

我们还可以使用运算符全名函数中的`out`参数或者自加运算符`+=`(也即`add_()`)达到上述效果.

In [14]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
id_before = id(y)
torch.add(x, y, out=y) # y += x, y.add_(x)
print(id(y) == id_before) # True

True


关于view():虽然view返回的Tensor与源Tensor是共享data的，但是依然是一个新的Tensor（因为Tensor除了包含data外还有一些其他属性），二者id（内存地址）并不一致。

# Tensor和Numpy相互转换

注意，用`numpy()`和`from_numpy()`将`Tensor`和`NumPy`中的数组相互转换。但是需要注意的一点是： 这两个函数所产生的`Tensor`和`NumPy`中的数组共享相同的内存（所以他们之间的转换很快），改变其中一个时另一个也会改变！！！

In [16]:
# tensor 转 numpy
a = torch.ones(5)
b = a.numpy()
print(a, b)

a+=1
print(a,b)
b+=2
print(a, b)

tensor([1., 1., 1., 1., 1.]) [1. 1. 1. 1. 1.]
tensor([2., 2., 2., 2., 2.]) [2. 2. 2. 2. 2.]
tensor([4., 4., 4., 4., 4.]) [4. 4. 4. 4. 4.]


# NumPy转Tensor

In [17]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
print(a, b)

a += 1
print(a, b)
b += 1
print(a, b)

[1. 1. 1. 1. 1.] tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
[2. 2. 2. 2. 2.] tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
[3. 3. 3. 3. 3.] tensor([3., 3., 3., 3., 3.], dtype=torch.float64)


所有在CPU上的Tensor（除了CharTensor）都支持与NumPy数组相互转换。

此外上面提到还有一个常用的方法就是直接用torch.tensor()将NumPy数组转换成Tensor，需要注意的是该方法总是会进行数据拷贝，**返回的Tensor和原来的数据不再共享内存**。

In [18]:
c = torch.tensor(a)
a += 1
print(a, c)

[4. 4. 4. 4. 4.] tensor([3., 3., 3., 3., 3.], dtype=torch.float64)


# Tensor on GPU

In [20]:
# 使用 to() 将 Tensor 在 CPU 和 GPU之间的移动
# 需要有硬件支持
print(torch.cuda.is_available())
if torch.cuda.is_available():
    device = torch.device("cuda")          # GPU
    y = torch.ones_like(x, device=device)  # 直接创建一个在GPU上的Tensor
    x = x.to(device)                       # 等价于 .to("cuda")
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # to()还可以同时更改数据类型

True
tensor([2, 3], device='cuda:0')
tensor([2., 3.], dtype=torch.float64)
